In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [3]:
df = pd.read_csv("final_df.csv")

In [4]:
df.head(250000)

,Unnamed: 0,text,sadness,joy,love,anger,fear,surprise,total
0,0,That game hurt.,1,0,0,0,0,0,1
1,1,Man I love reddit.,0,0,1,0,0,0,1
2,2,So happy for [NAME]. So sad he's not here. Ima...,1,1,0,0,0,0,2
3,3,"I just came home, what the fuck is this lineup...",0,0,1,0,0,0,1
4,4,By far the coolest thing I've seen on this thr...,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
249995,249995,i started the year plagued by this question if...,1,0,0,0,0,0,2
249996,249996,i feel really messy,1,0,0,0,0,0,2
249997,249997,i feel stupid for meeting him again even thoug...,1,0,0,0,0,0,2
249998,249998,i know that what im feeling isnt hunger but ra...,0,1,0,0,0,0,2


In [5]:
df = df.iloc[:100000]

In [6]:
# Drop unnecessary columns
df = df.drop(columns=["Unnamed: 0"])

# Separate features and labels
X = df["text"]
y = df.drop(columns=["text"])


In [ ]:
# Convert text data to numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Word embeddings using Word2Vec
word_embeddings_model = Word2Vec(sentences=X, vector_size=100, window=5, min_count=1, workers=4)
word_embeddings = np.array([np.mean([word_embeddings_model.wv[word] for word in sentence.split() if word in word_embeddings_model.wv] or [np.zeros(100)], axis=0) for sentence in X])

# Combine TF-IDF and word embeddings
X_combined = np.hstack((X_tfidf.toarray(), word_embeddings))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))
